<a href="https://colab.research.google.com/github/tavares51/agro/blob/main/analise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Criação da Base de Dados

In [2]:
vendedores = {1: 'João', 2: 'Maria', 3: 'Pedro', 4: 'Ana', 5: 'Carlos'}
# valores considerando a uma Saca (60kg)
produtos = [
    {'ProdutoId':1, 'Produto':'Soja', 'Valor': 136.42},
    {'ProdutoId':2, 'Produto':'Milho', 'Valor': 57.71},
    {'ProdutoId':3, 'Produto':'Sorgo', 'Valor': 38.00}
    ]
regioes = {1:'Norte', 2:'Sul', 3:'Leste', 4:'Oeste', 5:'Centro'}

In [3]:
vendedores_df = pd.DataFrame(list(vendedores.items()), columns=['VendedorId', 'Vendedor'])
produtos_df = pd.DataFrame(produtos)
regioes_df = pd.DataFrame(list(regioes.items()), columns=['RegiaoId', 'Regiao'])

In [4]:
clientes = list()
for i in range(1,16):
    clientes.append({'ClienteId':i, 'Cliente':f'Cliente {i}', 'VendedorId':(i % 5) + 1})
clientes_df = pd.DataFrame(clientes)

In [5]:
vendedores_df.head()

,VendedorId,Vendedor
0,1,João
1,2,Maria
2,3,Pedro
3,4,Ana
4,5,Carlos


In [6]:
produtos_df.head()

,ProdutoId,Produto,Valor
0,1,Soja,136.42
1,2,Milho,57.71
2,3,Sorgo,38.00


In [7]:
regioes_df.head()

,RegiaoId,Regiao
0,1,Norte
1,2,Sul
2,3,Leste
3,4,Oeste
4,5,Centro


In [8]:
clientes_df.head()

,ClienteId,Cliente,VendedorId
0,1,Cliente 1,2
1,2,Cliente 2,3
2,3,Cliente 3,4
3,4,Cliente 4,5
4,5,Cliente 5,1


Geração dos Dados de Venda

In [23]:
SEED = 42
random.seed(SEED)

data_inicial = datetime(2023, 1, 1)
data_final = datetime(2023, 12, 31)
quantidades = list(range(50, 200))
num_registros = 200

vendas = []
for _ in range(num_registros):
    data_venda = data_inicial + timedelta(days=random.randint(0, (data_final - data_inicial).days))
    cliente_id = random.randint(clientes_df.ClienteId.min(), clientes_df.ClienteId.max())
    vendedor_id = clientes_df.loc[clientes_df['ClienteId'] == cliente_id, 'VendedorId'].values[0]
    produto_id = random.choice([1, 2, 3])
    regiao_id = random.randint(1, 5)
    quantidade = random.choice(quantidades)
    vendas.append([cliente_id, vendedor_id, produto_id, data_venda, regiao_id, quantidade])

colunas = ["ClienteId", "VendedorId", "ProdutoId", "DataVenda", "RegiaoId", "Quantidade"]
df_vendas = pd.DataFrame(vendas, columns=colunas)

In [34]:
df_vendas = pd.DataFrame(vendas, columns=["ClienteId", "VendedorId", "ProdutoId", "DataVenda", "RegiaoId", "Quantidade"])

df_vendas = df_vendas.merge(vendedores_df, on="VendedorId")

df_vendas = df_vendas.merge(regioes_df, on="RegiaoId")

df_vendas = df_vendas.merge(produtos_df, on="ProdutoId")

df_vendas = df_vendas.merge(clientes_df, on="ClienteId")

df_vendas['Valor_Total'] = df_vendas['Quantidade'] * df_vendas['Valor']
df_vendas['Valor_Total'] = df_vendas['Valor_Total'].round(2)

df_final = df_vendas[['Produto','Valor','Quantidade','Valor_Total','DataVenda','Vendedor','Cliente','Regiao']]

In [35]:
df_vendas.head()

,ClienteId,VendedorId_x,ProdutoId,DataVenda,RegiaoId,Quantidade,Vendedor,Regiao,Produto,Valor,Cliente,VendedorId_y,Valor_Total
0,2,3,1,2023-11-24,3,112,Pedro,Leste,Soja,136.42,Cliente 2,3,15279.04
1,2,3,1,2023-09-02,4,175,Pedro,Oeste,Soja,136.42,Cliente 2,3,23873.50
2,2,3,1,2023-09-10,1,89,Pedro,Norte,Soja,136.42,Cliente 2,3,12141.38
3,2,3,2,2023-07-14,3,117,Pedro,Leste,Milho,57.71,Cliente 2,3,6752.07
4,2,3,2,2023-01-15,2,198,Pedro,Sul,Milho,57.71,Cliente 2,3,11426.58


# Vizulização de Dados

In [13]:
import plotly.express as px

In [36]:
df_regiao_clientes = df_vendas.groupby('RegiaoId')['ClienteId'].value_counts().reset_index(name='QuantidadeClientes')
df_regiao_clientes = df_regiao_clientes.merge(regioes_df, on='RegiaoId')
quantidade_cliente_regiao = df_regiao_clientes.Regiao.value_counts()

In [62]:
ax1 = px.bar(
    x=quantidade_cliente_regiao.values
    , y=quantidade_cliente_regiao.index
    , title='Quantidade de Clientes por Região'
    , labels={'x':'Região', 'y':'Quantidade de Clientes'}
    , color=quantidade_cliente_regiao.index
    , color_discrete_sequence=['#1f77b4', '#ffffff', '#2ca02c', '#d62728', '#9467bd']
    , width=500
    , height=350
    , text=quantidade_cliente_regiao.values
    , text_auto=True
    , template='plotly_dark'
    , opacity=0.7
    , orientation='h'
)
ax1.update_layout(title_x=0.5)
ax1.show()

In [61]:
meses = ['Janeiro','Fevereiro','Março','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro']
df_vendas_mes = df_vendas.copy()
df_vendas_mes['mes'] = df_vendas_mes['DataVenda'].dt.month
df_vendas_mes['ano'] = df_vendas_mes['DataVenda'].dt.year
df_vendas_mes = df_vendas_mes.groupby(['Vendedor', 'mes', 'ano'], as_index=False)['Valor_Total'].sum()
#mes_filtro = int(input('Insira o mês para filtrar (1-12): '))
mes_filtro = 2
meta_vendas = 100000
df_vendas_mes_filtrado = df_vendas_mes[df_vendas_mes['mes'] == mes_filtro]

ax2 = px.bar(
    df_vendas_mes_filtrado
    , x='Vendedor'
    , y='Valor_Total'
    , color='Vendedor'
    , color_discrete_sequence=['#1f77b4', '#ffffff', '#2ca02c', '#d62728', '#9467bd']
    , title='Valor Total de Vendas por Vendedor - Mês: ' + str(meses[mes_filtro-1])
    , labels={'x':'Vendedor', 'y':'Valor Total de Vendas'}
    , width=550
    , height=350
    , text='Valor_Total'
    , text_auto=True
    , template='plotly_dark'
    , opacity=0.7
    ,range_y = [0, meta_vendas]
    )
ax2.update_layout(title_x=0.5)
ax2.show()


In [59]:
df_vendas_mes_1 = df_vendas.copy()
df_vendas_mes_1['mes'] = df_vendas_mes_1['DataVenda'].dt.month
df_vendas_mes_1['ano'] = df_vendas_mes_1['DataVenda'].dt.year
df_vendas_mes_1 = df_vendas_mes_1.groupby(['ProdutoId', 'mes', 'ano'], as_index=False)['Valor_Total'].sum()
#mes_filtro = int(input('Insira o mês para filtrar (1-12): '))
mes_filtro = 2
meta_vendas = 100000
df_vendas_mes1_filtrado = df_vendas_mes_1[df_vendas_mes['mes'] == mes_filtro]
df_vendas_mes1_filtrado = df_vendas_mes1_filtrado.groupby('ProdutoId')['Valor_Total'].sum().reset_index()
df_vendas_mes1_filtrado = df_vendas_mes1_filtrado.merge(produtos_df, on='ProdutoId')
ax3 = px.bar(
    df_vendas_mes1_filtrado
    , x='Produto'
    , y='Valor_Total'
    , color='Produto'
    , color_discrete_sequence=['#1f77b4', '#ffffff', '#2ca02c', '#d62728', '#9467bd']
    , title='Quantidade Total de Vendas por Produto - Mês: ' + str(meses[mes_filtro-1])
    , labels={'x':'Produto', 'y':'Quantidade Total de Vendas'}
    , width=500
    , height=350
    , text='Valor'
    , text_auto=True
    , template='plotly_dark'
    , opacity=0.7
    )
ax3.update_layout(title_x=0.5)
ax3.show()


<ipython-input-59-da7b97c00b14>:8: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [63]:
df_vendas['mes'] = df_vendas['DataVenda'].dt.month
df_vendas_produto_mes = df_vendas.groupby(['mes', 'Vendedor'], as_index=False)['Valor_Total'].sum()
meses = ['Janeiro','Fevereiro','Março','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro']
ax4 = px.line(df_vendas_produto_mes, x="mes", y="Valor_Total", color='Vendedor', color_discrete_sequence=px.colors.qualitative.Pastel1)

ax4.update_layout(
    title="Vendas por Vendedor em 2023",
    xaxis_title="Mês",
    yaxis_title="Valor Total de Vendas",
    legend_title="Vendedor",
    template='plotly_dark',
    width=1500,
    height=400,
    font=dict(size=14),
    legend=dict(font=dict(size=12)),
    xaxis=dict(ticktext=meses, tickvals=list(range(1, 13))),
    yaxis=dict(tickformat=".2f"),
    hovermode="x",
    hoverlabel=dict(font=dict(size=12)),
    margin=dict(l=50, r=50, t=50, b=50),
    showlegend=True,
    title_x=0.5,
    title_font=dict(size=20)
)
ax4.add_hline(y=50000, line_dash="dash", line_color="red", annotation=dict(text="Meta de Vendas", showarrow=False))
ax4.show()